# Wordle Solver

We'll start with a dumb solver and work out way up to something nicer.

Let's start with some dependencies we need to make this work.

In [74]:
USE {
    dependencies {
    }
}

First we need to "model" the game. The game state is a list of guesses, where each guess is a five-letter word and each letter is colored yellow, green or red depending on whether the letter is in the word, in the right place, or not in the word at all. We'll represent this as a list of tuples, where each tuple is a letter and a color.

Sealed classes seem like a nice way to model colored letters, let's also write some `toString` implementations so we can print them out nicely.

In [75]:
sealed class Letter {
    abstract val char: Char

    protected fun toString(color: String) = "<td style=\"background-color: $color\">$char</td>"

    data class Yellow(override val char: Char) : Letter() {
        override fun toString() = toString("yellow")
    }

    data class Green(override val char: Char) : Letter() {
        override fun toString() = toString("green")
    }

    data class Red(override val char: Char) : Letter() {
        override fun toString() = toString("red")
    }
}

Let's test it out

In [76]:
HTML("<table><tr>${listOf(Letter.Green('a'), Letter.Yellow('b'), Letter.Red('c')).joinToString("")}</tr></table>")

a,b,c


Excellent, now let's model the game state


In [77]:
data class Game(val guesses: List<List<Letter>>) {
    init {
        require(guesses.size <= 5) { "Game must have 5 guesses at most" }
        require(guesses.all { it.size == 5 }) { "Each guess must have 5 letters" }
    }

    override fun toString() = "<table>${guesses.joinToString("") { "<tr>${it.joinToString("")}</tr>" }}</table>"
}

Let's test it out

In [78]:
HTML(
    Game(
        listOf(
            listOf(Letter.Green('a'), Letter.Yellow('b'), Letter.Red('c'), Letter.Green('d'), Letter.Yellow('e')),
            listOf(Letter.Green('v'), Letter.Yellow('w'), Letter.Red('x'), Letter.Green('y'), Letter.Yellow('z'))
        )
    ).toString()
)

a,b,c,d,e
v,w,x,y,z


Let's define a "guesser" abstraction and a dummy implementation so we can write a game driver around it.

In [79]:
typealias Guesser = (Game) -> String

val staticGuesser: Guesser = { "abcde" }

Let's write a game driver that takes a guesser and runs a game until it's over.

In [80]:
fun playGame(actualWord: String, guesser: Guesser): Game {
    var game = Game(emptyList())
    var solved = false
    while (game.guesses.size < 5 && !solved) {
        val guess = guesser(game)
        check(guess.length == 5) { "Guess must be 5 letters" }

        val letters = guess.mapIndexed { index, char ->
            when {
                char == actualWord[index] -> Letter.Green(char)
                char in actualWord -> Letter.Yellow(char)
                else -> Letter.Red(char)
            }
        }

        game = game.copy(guesses = game.guesses + listOf(letters))
        solved = letters.all { it is Letter.Green }
    }

    return game
}

Let's play a game with our dummy guesser and make it win

In [81]:
HTML(playGame("abcde", staticGuesser).toString())

a,b,c,d,e


Now make it lose

In [82]:
HTML(playGame("awcye", staticGuesser).toString())

a,b,c,d,e
a,b,c,d,e
a,b,c,d,e
a,b,c,d,e
a,b,c,d,e


To write better guessers we need to know the dictionary of valid words. Let's load that up.

In [83]:
val dictionary = java.io.File("/usr/share/dict/words")
    .readLines()
    .filter { it.length == 5 }
    .filter { it.all { char -> char in 'a'..'z' } }
    .filter { it.toSet().size == 5 }

println("loaded ${dictionary.size} words")


loaded 5497 words


We also need to get more useful information out of the game state, it's not easy to work with nested lists all the time, so let's write some helper functions to make it easier to work with.


In [84]:
// just the words
fun Game.words() = guesses.map { it.joinToString("") { letter -> letter.char.toString() } }

// all red letters
fun Game.redLetters() = guesses.flatMap { it.filterIsInstance<Letter.Red>() }
    .map { it.char }
    .toSet()

// all green letters but we need to know the positions too
fun Game.greenLetters() = guesses
    .map { it.withIndex() }
    .flatMap { it.filter { (_, letter) -> letter is Letter.Green } }
    .distinctBy { it.value.char }
    .associate { it.value.char to it.index }

// all yellow letters, except those that turned out green later.
// we also need positions for these but remember that a yellow letter can be in multiple positions
fun Game.yellowLetters(): Map<Char, Set<Int>> = guesses
    .map { it.withIndex() }
    .flatMap { it.filter { (_, letter) -> letter is Letter.Yellow } }
    .groupBy { it.value.char }
    .mapValues { it.value.map { it.index }.toSet() }
    .minus(greenLetters().keys)

// let's see if it works
fun gameFunctionsTest() {
    val game = Game(
        listOf(
            listOf(Letter.Green('a'), Letter.Yellow('b'), Letter.Red('c'), Letter.Green('d'), Letter.Yellow('e')),
            listOf(Letter.Green('a'), Letter.Green('e'), Letter.Yellow('b'), Letter.Green('d'), Letter.Red('f'))
        )
    )
    println("red: ${game.redLetters()}")
    println("green: ${game.greenLetters()}")
    println("yellow: ${game.yellowLetters()}")
}

gameFunctionsTest()

red: [c, f]
green: {a=0, d=3, e=1}
yellow: {b=[1, 2]}


With that we can now write a better guesser that finds the first word in the dictionary that has none of the red letters, all the green letters at the same positions, and all the yellow letters in any position. For this iteration we will not take into account the positions of the yellow letters.


In [85]:
fun simpleFirstMatchGuesser(dictionary: List<String>, debug: Boolean = false): Guesser = { game ->
    val redLetters = game.redLetters()
    val greenLetters = game.greenLetters()
    val yellowLetters = game.yellowLetters()
    val usedWords = game.words()

    fun Sequence<String>.f(name: String, fn: (String) -> Boolean) = filter {
        val result = fn(it)
        if (debug) {
            println("? $name: $it -> $result")
        }
        result
    }

    dictionary
        .asSequence()
        .f("used") { word -> word !in usedWords }
        .f("red($redLetters)") { word -> word.none { it in redLetters } }
        .f("green($greenLetters)") { word -> greenLetters.all { (char, index) -> word[index] == char } }
        .f("yellow($yellowLetters)") { word -> yellowLetters.all { (char, _) -> char in word } }
        .first()
}

// let's test it out
fun testSimpleFirstMatchGuesser() {
    val simpleDict = listOf("abcde", "xxxxx", "afcge", "yyyyy", "agcfe", "zzzzz")
    val guesser = simpleFirstMatchGuesser(simpleDict, debug = true)

    // shortcuts!
    fun g(char: Char) = Letter.Green(char)
    fun y(char: Char) = Letter.Yellow(char)
    fun r(char: Char) = Letter.Red(char)
    val (a, b, c, d, e) = ('a'..'e').toList()
    val (f, g) = ('f'..'g').toList()

    fun game(vararg letters: Letter) = Game(letters.toList().chunked(5))

    val firstGuess = guesser(game())
    println("> first guess: $firstGuess")
    check(firstGuess == "abcde") { "first guess should be abcde, was $firstGuess" }

    val secondGuess = guesser(
        game(
            g(a), r(b), g(c), r(d), g(e)
        )
    )
    println("> second guess: $secondGuess")
    check(secondGuess == "afcge") { "second guess should be afcge, was $secondGuess" }

    val thirdGuess = guesser(
        game(
            g(a), r(b), g(c), r(d), y(e),
            g(a), y(f), g(c), y(g), g(e)
        )
    )

    println("> third guess: $thirdGuess")
    check(thirdGuess == "agcfe") { "third guess should be agcfe, was $thirdGuess" }
}

testSimpleFirstMatchGuesser()


? used: abcde -> true
? red([]): abcde -> true
? green({}): abcde -> true
? yellow({}): abcde -> true
> first guess: abcde
? used: abcde -> false
? used: xxxxx -> true
? red([b, d]): xxxxx -> true
? green({a=0, c=2, e=4}): xxxxx -> false
? used: afcge -> true
? red([b, d]): afcge -> true
? green({a=0, c=2, e=4}): afcge -> true
? yellow({}): afcge -> true
> second guess: afcge
? used: abcde -> false
? used: xxxxx -> true
? red([b, d]): xxxxx -> true
? green({a=0, c=2, e=4}): xxxxx -> false
? used: afcge -> false
? used: yyyyy -> true
? red([b, d]): yyyyy -> true
? green({a=0, c=2, e=4}): yyyyy -> false
? used: agcfe -> true
? red([b, d]): agcfe -> true
? green({a=0, c=2, e=4}): agcfe -> true
? yellow({f=[1], g=[3]}): agcfe -> true
> third guess: agcfe


Sweet, so now we have a guesser that at least makes an attempt to win the game, but we don't know how _good_ it is. Let's write a benchmark function that plays a game a bunch of times and collects statistics about the number of guesses it took to win. In order to make comparisons between guessers easier we will also make sure that the same words are used in each game.


In [87]:
import java.math.BigDecimal
import java.math.RoundingMode

val random = Random(0)
val benchmarkWords = dictionary.shuffled(random).take(100)

sealed class Outcome {
    object Lost : Outcome()
    data class Won(val guesses: Int) : Outcome()
}

fun Game.outcome() = if (guesses.last().all { it is Letter.Green }) {
    Outcome.Won(guesses.size)
} else {
    Outcome.Lost
}

data class Statistics(
    val percentWon: BigDecimal,
    val guessFrequency: Map<Int, BigDecimal>
)

fun benchmark(guesser: Guesser, words: List<String> = benchmarkWords): Statistics {
    val outcomes = words.map { word ->
        playGame(word, guesser).outcome()
    }

    val total = outcomes.size
    val numWon = outcomes.count { it is Outcome.Won }
    val percentWon = numWon.toDouble() / total * 100

    val guessFrequency = outcomes.filterIsInstance<Outcome.Won>()
        .map { it.guesses }
        .groupingBy { it }
        .eachCount()
        .mapValues { it.value.toDouble() / total * 100 }
        .toSortedMap()

    fun Double.round() = BigDecimal(this).setScale(2, RoundingMode.HALF_EVEN)

    return Statistics(percentWon.round(), guessFrequency.mapValues { it.value.round() })
}

Right, now let's benchmark the simple guesser.


In [91]:
%use kandy
import org.w3c.dom.Document

val simpleGuesserStats = benchmark(simpleFirstMatchGuesser(dictionary))
println("Simple Guesser won ${simpleGuesserStats.percentWon}% of games!")
plot {
    bars {
        x(simpleGuesserStats.guessFrequency.keys)
        y(simpleGuesserStats.guessFrequency.values)
    }
}


Simple Guesser won 55.00% of games!


That's pretty cool! The simple guesser won over half the games and has a nice linear trend line on the guess frequency.

We can however, do better. Note that the simple guesser does not take into account the positions of the yellow letters. Let's write a guesser that does and benchmark it.


In [90]:
fun yellowPostionAwareGuesser(dictionary: List<String>, debug: Boolean = false): Guesser = { game ->
    val redLetters = game.redLetters()
    val greenLetters = game.greenLetters()
    val yellowLetters = game.yellowLetters()
    val usedWords = game.words()

    fun Sequence<String>.f(name: String, fn: (String) -> Boolean) = filter {
        val result = fn(it)
        if (debug) {
            println("? $name: $it -> $result")
        }
        result
    }

    dictionary
        .asSequence()
        .f("used") { word -> word !in usedWords }
        .f("red($redLetters)") { word -> word.none { it in redLetters } }
        .f("green($greenLetters)") { word -> greenLetters.all { (char, index) -> word[index] == char } }
        .f("yellow($yellowLetters)") { word ->
            yellowLetters.all { (char, indexes) ->
                char in word && indexes.all { index -> word[index] != char }
            }
        }
        .first()
}

val yellowPositionAwareGuesserStats = benchmark(yellowPostionAwareGuesser(dictionary))
println("Yellow Position Aware Guesser won ${yellowPositionAwareGuesserStats.percentWon}% of games!")

plot {
    bars {
        x(yellowPositionAwareGuesserStats.guessFrequency.keys)
        y(yellowPositionAwareGuesserStats.guessFrequency.values)
    }
}


Yellow Position Aware Guesser won 82.00% of games!


That is a significant improvement! The yellow position aware guesser wins over 80% of games and has a much nicer guess frequency distribution: in fact it wins most games on or before the 4th guess.
